## 1_でクロールしたpicleファイルをロードし、一つにまとめて、解析に使いやすいようにデータを整形したうえで再度pickleファイルで保存
- ファイルの読み込み元：'./crawledData/　以下
- ファイルの保存先：'./arrangedData/　以下

In [1]:
import glob
import pandas as pd

#### レース情報をまとめたpicleファイルを読み込んで一つにまとめる

In [2]:
# ファイル内容確認用
all_files = glob.glob('./data/crawledData/*.pkl')

# その日のレース情報をまとめたdfを要素にもつリスト
race_df_list = [pd.read_pickle(file) for file in all_files]
# concatして一つのdfにまとめる
race_df = pd.concat(race_df_list)
race_df

boatNo motorNo num_false_start num_late_start  \
date       venue raceNumber 枠                                                 
2021-02-18 桐　生   1          1     29      49              F0             L0   
                            2     47      54              F0             L0   
                            3     70      19              F0             L0   
                            4     71      39              F0             L0   
                            5     25      24              F0             L0   
...                              ...     ...             ...            ...   
2021-02-15 唐　津   12         2     60      36              F0             L0   
                            3     73      19              F0             L0   
                            4     77      18              F0             L0   
                            5     86      33              F1             L0   
                            6     84      27              F1             L0   

                              racer_class racer_id racer_name  ボート2連率  ボート3連率  \
date       venue raceNumber 枠                                                   
2021-02-18 桐　生   1          1          B1     4078     渡辺　　史之   44.90   53.06   
                            2          B2     3227     長岡　　茂一   32.69   42.31   
                            3          B1     5049     松井　　友汰   28.00   42.00   
                            4          A1     4131     原　　　豊土   46.94   63.27   
                            5          B1     3748     小黒　　竜吾   21.74   36.96   
...                                   ...      ...        ...     ...     ...   
2021-02-15 唐　津   12         2          A2     4730     土屋　実沙希   30.96   49.75   
                            3          B2     4764     小池　　礼乃   35.24   48.10   
                            4          B1     4726     島田　なぎさ   26.90   43.86   
                            5          A1     3900     香川　　素子   33.87   53.23   
                            6          A2     4349     犬童　　千秋   36.90   50.27   

                              モーター2連率  ... late temperature  tilt  \
date       venue raceNumber 枠          ...                          
2021-02-18 桐　生   1          1   25.00  ...    0         0.0   0.0   
                            2   24.24  ...    0         0.0   0.0   
                            3   20.00  ...    0         0.0   0.0   
                            4   32.14  ...    0         0.0   0.0   
                            5   48.28  ...    0         0.0   0.0   
...                               ...  ...  ...         ...   ...   
2021-02-15 唐　津   12         2   47.41  ...    0        13.0  -0.5   
                            3   30.00  ...    0        13.0  -0.5   
                            4   33.33  ...    0        13.0  -0.5   
                            5   44.83  ...    0        13.0  -0.5   
                            6   48.15  ...    0        13.0  -0.5   

                              water_temperature  wave_height  weather weight  \
date       venue raceNumber 枠                                                  
2021-02-18 桐　生   1          1               9.0            2        晴   50.5   
                            2               9.0            2        晴   52.5   
                            3               9.0            2        晴   53.2   
                            4               9.0            2        晴   52.1   
                            5               9.0            2        晴   55.9   
...                                         ...          ...      ...    ...   
2021-02-15 唐　津   12         2              11.0            5        晴   46.5   
                            3              11.0            5        晴   46.5   
                            4              11.0            5        晴   46.0   
                            5              11.0            5        晴   47.0   
                            6              11.0            5        晴   48.1   

         

#### 解析に使いやすいようにデータを整形する

In [11]:
from datetime import datetime as dt

race_df_arranged = race_df.copy()

# それぞれの列のデータタイプの変更など
race_df_arranged["boatNo"] = race_df_arranged["boatNo"].astype("int")
race_df_arranged["motorNo"] = race_df_arranged["motorNo"].astype("int")
race_df_arranged["num_false_start"] = race_df_arranged["num_false_start"].map(lambda x: int(str(x)[1]))
race_df_arranged["num_late_start"] = race_df_arranged["num_late_start"].map(lambda x: int(str(x)[1]))
race_df_arranged["racer_id"] = race_df_arranged["racer_id"].astype("int")
race_df_arranged["ボート2連率"] = race_df_arranged["ボート2連率"].astype("float64")
race_df_arranged["ボート3連率"] = race_df_arranged["ボート3連率"].astype("float64")
race_df_arranged["racer_class"] = race_df_arranged["racer_class"].map({"A1": 1, "A2": 2, "B1": 3, "B2": 4})
race_df_arranged["モーター2連率"] = race_df_arranged["モーター2連率"].astype("float64")
race_df_arranged["モーター3連率"] = race_df_arranged["モーター3連率"].astype("float64")
race_df_arranged["exhibition_ST"] = race_df_arranged["exhibition_ST"].astype("float64")
race_df_arranged["tilt"] = race_df_arranged["tilt"].astype("float64")
race_df_arranged["weight"] = race_df_arranged["weight"].astype("float64")

# weatherをdummy変数に変換
dummy_df_weather = pd.get_dummies(race_df_arranged["weather"], prefix='weather', drop_first=True)
race_df_arranged = pd.concat([race_df_arranged, dummy_df_weather], axis=1)

# 何らかの事情でデータから抜いたほうがいいレースを集める
race_df_del_list = []
# 転覆等があり、着順が6までつけられなかったレースを削除. 転覆になった選手の行だけではなくそのレースごと列を削除する
race_df_del_1 = race_df_arranged[race_df_arranged["着順"].astype(str).str.contains("\D")]
race_df_del_1.reset_index(inplace=True)
race_df_del_list.append(race_df_del_1)

# それぞれvalueに''を含むレースを削除
may_have_blank_list = [
    "exhibitionTime", "temperature", "water_temperature",
    "wave_height", "wind_speed", "exhibition_cource"
]
for column in may_have_blank_list:
    race_df_del_ = race_df_arranged[~race_df_arranged[column].str.match("\d.*")]
    race_df_del_.reset_index(inplace=True)
    race_df_del_list.append(race_df_del_)

# Nanを含む列があるレースを削除
race_df_del_nan = race_df_arranged[race_df_arranged.isnull().any(axis=1)]
race_df_del_nan.reset_index(inplace=True)
race_df_del_list.append(race_df_del_nan)

# 消したいレースの開催地・場所・レース番号のリストを作成
remove_race_list_nested = [race_df_del[["date", "venue", "raceNumber"]].values.tolist() for race_df_del in race_df_del_list]
remove_race_list =  [item for l in remove_race_list_nested for item in l]

remove_index_list = list(set([tuple(race) for race in remove_race_list]))
race_df_arranged.drop(index=remove_index_list, inplace=True)

# 着順のカラムをinterger型に変換
race_df_arranged["着順"] = race_df_arranged["着順"].astype("int")

race_df_arranged["exhibitionTime"] = race_df_arranged["exhibitionTime"].astype("float64")
race_df_arranged["temperature"] = race_df_arranged["temperature"].astype("float64")
race_df_arranged["water_temperature"] = race_df_arranged["water_temperature"].astype("float64")
race_df_arranged["wave_height"] = race_df_arranged["wave_height"].astype("int")
race_df_arranged["wind_speed"] = race_df_arranged["wind_speed"].astype("int")
race_df_arranged["exhibition_cource"] = race_df_arranged["exhibition_cource"].astype("int")

# multiindexは扱いがめんどくさいので全部カラムにする
race_df_arranged.reset_index(inplace=True)

# date列をdatetim型に変換
race_df_arranged["date"] = race_df_arranged["date"].map(lambda x: dt.strptime(x, '%Y-%m-%d'))

# dateとraceNumberでsort
race_df_arranged["raceNumber"] = race_df_arranged["raceNumber"].astype("float")
race_df_arranged.sort_values(["date", "venue", "raceNumber", "枠"], inplace=True)

# 統計量の算出を行うためのdf: pickleファイルで保存
race_df_arranged.to_pickle('./data/arrangedData/data_formatted_for_statistic.pkl')

# 予測の際にそのままinputにすることができない列を削除
race_df_arranged.drop(["boatNo", "motorNo", "racer_id", "racer_name", "weather", "タイム"], axis=1, inplace=True)

# 予測用のdf: pickleファイルで保存
race_df_arranged.to_pickle('./data/arrangedData/data_formatted_1.pkl')

/Users/grice/opt/anaconda3/envs/boatrace2/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


In [12]:
race_df_arranged

,date,venue,raceNumber,枠,num_false_start,num_late_start,racer_class,ボート2連率,ボート3連率,モーター2連率,...,temperature,tilt,water_temperature,wave_height,weight,wind_speed,着順,weather_曇り,weather_雨,weather_雪
38706,2021-02-01,津,1.0,1,0,0,1,38.46,55.77,36.59,...,8.0,-0.5,6.0,1,52.2,1,1,1,0,0
38707,2021-02-01,津,1.0,2,0,0,3,41.25,60.00,27.93,...,8.0,-0.5,6.0,1,53.1,1,2,1,0,0
38708,2021-02-01,津,1.0,3,0,0,3,25.56,55.56,33.33,...,8.0,-0.5,6.0,1,53.2,1,3,1,0,0
38709,2021-02-01,津,1.0,4,0,0,3,24.66,45.21,37.36,...,8.0,-0.5,6.0,1,52.7,1,4,1,0,0
38710,2021-02-01,津,1.0,5,0,0,4,40.58,57.97,37.39,...,8.0,-0.5,6.0,1,52.9,1,5,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12667,2021-03-21,蒲 郡,12.0,2,0,0,1,27.49,46.20,33.33,...,13.0,-0.5,16.0,3,52.5,5,2,1,0,0
12668,2021-03-21,蒲 郡,12.0,3,0,0,1,28.75,40.63,50.29,...,13.0,-0.5,16.0,3,52.3,5,5,1,0,0
12669,2021-03-21,蒲 郡,12.0,4,0,0,1,34.83,48.76,34.76,...,13.0,-0.5,16.0,3,55.9,5,6,1,0,0
12670,2021-03-21,蒲 郡,12.0,5,1,0,1,27.68,47.46,35.20,...,13.0,-0.5,16.0,3,53.7,5,3,1,0,0
